<a href="https://colab.research.google.com/github/CAMI2-bit/SIMULACION-I/blob/main/INVENTARIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
from itertools import product

prob_demanda = {
    35: 0.010, 36: 0.015, 37: 0.020, 38: 0.020, 39: 0.022,
    40: 0.023, 41: 0.025, 42: 0.027, 43: 0.028, 44: 0.029,
    45: 0.035, 46: 0.045, 47: 0.060, 48: 0.065, 49: 0.070,
    50: 0.080, 51: 0.075, 52: 0.070, 53: 0.065, 54: 0.060,
    55: 0.050, 56: 0.040, 57: 0.030, 58: 0.016, 59: 0.015,
    60: 0.005
}

#Probabilidades del tiempo de entrega (en meses)
tiempos_entrega = {1: 0.3, 2: 0.4, 3: 0.3}

#Ajustes estacionales por mes
ajuste_mes = {
    1: 1.20, 2: 1.00, 3: 0.90, 4: 0.80, 5: 0.80,
    6: 0.70, 7: 0.80, 8: 0.90, 9: 1.00, 10: 1.20,
    11: 1.30, 12: 1.40
}

#Costos unitarios
costo_pedido = 100
costo_almacenar = 20
costo_quiebre = 50

# Generador de demanda ajustada
def generar_demanda(mes):
    base = np.random.choice(list(prob_demanda.keys()), p=list(prob_demanda.values()))
    return int(base * ajuste_mes[mes])

#Simula un año de operación para un par (q, r)
def ejecutar_simulacion(q, r, stock_inicial=150, meses=12, mostrar=False):
    stock = stock_inicial
    pedidos = {}
    totales = {'pedidos': 0, 'almacenaje': 0, 'faltas': 0}

    for m in range(1, meses + 1):
        if m in pedidos:
            stock += pedidos.pop(m)

        demanda = generar_demanda(m)
        faltantes = max(0, demanda - stock)
        final_mes = max(0, stock - demanda)

        if faltantes > 0:
            inventario_medio = (stock**2) / (2 * demanda)
        else:
            inventario_medio = (stock + final_mes) / 2

        totales['almacenaje'] += inventario_medio * (costo_almacenar / 12)
        totales['faltas'] += faltantes * costo_quiebre

        if final_mes <= r and not any(k > m for k in pedidos.keys()):
            entrega = np.random.choice(list(tiempos_entrega.keys()), p=list(tiempos_entrega.values()))
            llegada = m + entrega
            pedidos[llegada] = q
            totales['pedidos'] += costo_pedido

        if mostrar:
            print(f"Mes {m}: Demanda={demanda}, Stock Inicial={stock}, "
                  f"Faltantes={faltantes}, Stock Final={final_mes}, Pedidos={pedidos}")

        stock = final_mes

    return sum(totales.values()), totales

#Prueba muchas combinaciones (q, r)
def optimizar_parametros(q_vals, r_vals, iteraciones=30):
    mejor_qr = None
    menor_costo = float('inf')
    historial = []

    for q, r in product(q_vals, r_vals):
        acumulado = 0
        for _ in range(iteraciones):
            total, _ = ejecutar_simulacion(q, r)
            acumulado += total
        promedio = acumulado / iteraciones
        historial.append((q, r, promedio))

        if promedio < menor_costo:
            menor_costo = promedio
            mejor_qr = (q, r)

    return mejor_qr, menor_costo, historial

#Punto de entrada principal
if __name__ == "__main__":
    q_opciones = range(150, 250, 10)
    r_opciones = range(80, 180, 10)

    (q_mejor, r_mejor), costo_minimo, todos = optimizar_parametros(q_opciones, r_opciones)

    print("\nParámetros óptimos:")
    print(f"→ q óptimo: {q_mejor}")
    print(f"→ r óptimo: {r_mejor}")
    print(f"→ Costo anual promedio: ${costo_minimo:.2f}")

    print("\nSimulación con valores óptimos:")
    total_final, desglose = ejecutar_simulacion(q_mejor, r_mejor, mostrar=True)
    print("\nDetalle de costos:")
    print(f"Pedidos realizados: ${desglose['pedidos']:.2f}")
    print(f"Costo por inventario: ${desglose['almacenaje']:.2f}")
    print(f"Costo por faltantes: ${desglose['faltas']:.2f}")
    print(f"Total anual: ${total_final:.2f}")



Parámetros óptimos:
→ q óptimo: 150
→ r óptimo: 120
→ Costo anual promedio: $3132.04

Simulación con valores óptimos:
Mes 1: Demanda=49, Stock Inicial=150, Faltantes=0, Stock Final=101, Pedidos={np.int64(3): 150}
Mes 2: Demanda=44, Stock Inicial=101, Faltantes=0, Stock Final=57, Pedidos={np.int64(3): 150}
Mes 3: Demanda=54, Stock Inicial=207, Faltantes=0, Stock Final=153, Pedidos={}
Mes 4: Demanda=35, Stock Inicial=153, Faltantes=0, Stock Final=118, Pedidos={np.int64(7): 150}
Mes 5: Demanda=44, Stock Inicial=118, Faltantes=0, Stock Final=74, Pedidos={np.int64(7): 150}
Mes 6: Demanda=37, Stock Inicial=74, Faltantes=0, Stock Final=37, Pedidos={np.int64(7): 150}
Mes 7: Demanda=31, Stock Inicial=187, Faltantes=0, Stock Final=156, Pedidos={}
Mes 8: Demanda=36, Stock Inicial=156, Faltantes=0, Stock Final=120, Pedidos={np.int64(10): 150}
Mes 9: Demanda=49, Stock Inicial=120, Faltantes=0, Stock Final=71, Pedidos={np.int64(10): 150}
Mes 10: Demanda=69, Stock Inicial=221, Faltantes=0, Stock Fin